In [20]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchinfo import summary
from torchmetrics import Accuracy
from torchvision import datasets
from torchvision.transforms import ToTensor

import mlflow
from mlflow.models import infer_signature, ModelSignature
from mlflow.types import Schema, TensorSpec

import optuna
from functools import partial

Let's load our training data FashionMNIST from torchvision, which has already been preprocessed into scale the [0, 1]. We then wrap the dataset into an instance of torch.utils.data.Dataloader.

training_data = datasets.FashionMNIST(root="data",  train=True,  download=True,  transform=ToTensor())

test_data = datasets.FashionMNIST(root="data",  train=False,  download=True,  transform=ToTensor())

Since we have already the data earlier, we will reuse the same, as below

In [2]:
# Download the training data from open datasets
training_data = datasets.FashionMNIST(root="/Users/debajyotidas/Library/CloudStorage/OneDrive-Personal/Online Courses/PyTorch/FashionMNIST", train=True, download=True, transform=ToTensor())

# Download the test data from open datasets
test_data = datasets.FashionMNIST(root="/Users/debajyotidas/Library/CloudStorage/OneDrive-Personal/Online Courses/PyTorch/FashionMNIST",train=False, download=True, transform=ToTensor())

In [3]:
print(f"Image size: {training_data[0][0].shape}")
print(f"Size of training dataset: {len(training_data)}")
print(f"Size of test dataset: {len(test_data)}")

Image size: torch.Size([1, 28, 28])
Size of training dataset: 60000
Size of test dataset: 10000


In [4]:
batch_size = 64
loss_fn = nn.CrossEntropyLoss() # Use CrossEntropyLoss when model outputs logits

if torch.cuda.is_available():
    device = torch.device("cuda")
    device_name = torch.cuda.get_device_name(0)
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

# Alternative using accelerate library if PyTorch version >= 2.6
# device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using mps device


In [5]:
# We wrap the dataset a Dataloader instance for batching purposes. Dataloader is a useful tool for data preprocessing.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [6]:
mlflow.login()

2025/12/28 00:41:16 INFO mlflow.utils.credentials: Successfully connected to MLflow hosted tracking server! Host: https://dbc-e4fb7400-b637.cloud.databricks.com.


In [8]:
#mlflow.set_tracking_uri("databricks")  # Telling MLflow to send the data into Databricks Workspace

mlflow.set_experiment("/Users/debajyoti.das.bookworm@gmail.com/mlflow-pytorch-quickstart")


# In order to keep all of your artifacts within a single place, we can opt to use Unity Catalog's Volumes feature. Firstly, you need to create a Unity Catalog Volume (let's call it: test.mlflow.mlflow-pytorch-quickstart). Then, we can run the following code to start an experiment with the Unity Catalog Volume and log metrics to it. Note that your experiment name must follow the /Users/<your email>/<experiment_name> format when using a Databricks Workspace.

#mlflow.create_experiment("/Users/debajyoti.das.bookworm@gmail.com/mlflow-pytorch-quickstart",artifact_location="dbfs:/Volumes/test/mlflow/mlflow-pytorch-quickstart")
#mlflow.set_experiment("/Users/debajyoti.das.bookworm@gmail.com/mlflow-pytorch-quickstart")

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/2972025542579128', creation_time=1766783078458, experiment_id='2972025542579128', last_update_time=1766873123972, lifecycle_stage='active', name='/Users/debajyoti.das.bookworm@gmail.com/mlflow-pytorch-quickstart', tags={'mlflow.experiment.sourceName': '/Users/debajyoti.das.bookworm@gmail.com/mlflow-pytorch-quickstart',
 'mlflow.experimentKind': 'custom_model_development',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'debajyoti.das.bookworm@gmail.com',
 'mlflow.ownerId': '8680600426295472'}>

In [6]:
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(nn.Conv2d(1, 8, kernel_size=3),
                                   nn.ReLU(),
                                   nn.Conv2d(8, 16, kernel_size=3),
                                   nn.ReLU(),
                                   nn.Flatten(),
                                   nn.LazyLinear(10),
                                   )

    def forward(self, x):
        return self.model(x)   

In [9]:
def train(dataloader, model, loss_fn, metrics_fn, optimizer, epoch):
  """Train the model on a single pass of the dataloader.

  Args:
      dataloader: an instance of `torch.utils.data.DataLoader`, containing the training data.
      model: an instance of `torch.nn.Module`, the model to be trained.
      loss_fn: a callable, the loss function.
      metrics_fn: a callable, the metrics function.
      optimizer: an instance of `torch.optim.Optimizer`, the optimizer used for training.
      epoch: an integer, the current epoch number.
  """
  model.train()
  for batch, (X, y) in enumerate(dataloader):
      X = X.to(device)
      y = y.to(device)

      pred = model(X)
      loss = loss_fn(pred, y)
      accuracy = metrics_fn(pred, y)

      # Backpropagation.
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()

      if batch % 100 == 0:
          loss_value = loss.item()
          current = batch
          step = batch // 100 * (epoch + 1)
          mlflow.log_metric("loss", f"{loss_value:2f}", step=step)
          mlflow.log_metric("accuracy", f"{accuracy:2f}", step=step)
          print(f"loss: {loss_value:2f} accuracy: {accuracy:2f} [{current} / {len(dataloader)}]")

In [10]:
def evaluate(dataloader, model, loss_fn, metrics_fn, epoch):
  """Evaluate the model on a single pass of the dataloader.

  Args:
      dataloader: an instance of `torch.utils.data.DataLoader`, containing the eval data.
      model: an instance of `torch.nn.Module`, the model to be trained.
      loss_fn: a callable, the loss function.
      metrics_fn: a callable, the metrics function.
      epoch: an integer, the current epoch number.
  """
  num_batches = len(dataloader)
  model.eval()
  eval_loss = 0
  eval_accuracy = 0
  with torch.no_grad():
      for X, y in dataloader:
          X = X.to(device)
          y = y.to(device)
          pred = model(X)
          eval_loss += loss_fn(pred, y).item()
          eval_accuracy += metrics_fn(pred, y)

  eval_loss /= num_batches
  eval_accuracy /= num_batches
  mlflow.log_metric("eval_loss", f"{eval_loss:2f}", step=epoch)
  mlflow.log_metric("eval_accuracy", f"{eval_accuracy:2f}", step=epoch)

  print(f"Eval metrics: Accuracy: {eval_accuracy:.2f}, Avg loss: {eval_loss:2f} ")

In [ ]:
epochs = 3
metric_fn = Accuracy(task="multiclass", num_classes=10).to(device)
model = ImageClassifier().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [16]:
# Define input and output schemas, in case infer_signature is not working as desired
input_schema = Schema([TensorSpec(np.dtype(np.float32), (1, 1, 28, 28))])
output_schema = Schema([TensorSpec(np.dtype(np.float32), (1, 1, 10))])
signature = ModelSignature(inputs=input_schema, outputs=output_schema)

In [ ]:
with mlflow.start_run() as run:
    params = {"epochs": epochs,
              "learning_rate": 1e-3,
              "batch_size": batch_size,
              "loss_function": loss_fn.__class__.__name__,
              "metric_function": metric_fn.__class__.__name__,
              "optimizer": "SGD"}
    
    # Log training parameters.
    mlflow.log_params(params)

    # Log model summary.
    with open("/Users/debajyotidas/Library/CloudStorage/OneDrive-Personal/Online Courses/MLFlow/model_summary.txt", "w") as f:
        f.write(str(summary(model)))
    mlflow.log_artifact("/Users/debajyotidas/Library/CloudStorage/OneDrive-Personal/Online Courses/MLFlow/model_summary.txt")
    
    for t in range(epochs):
        print(f"Epoch {t + 1}-------------------------------")
        train(train_dataloader, model, loss_fn, metric_fn, optimizer, epoch=t)
        evaluate(test_dataloader, model, loss_fn, metric_fn, epoch=0)

    # Build model signature by inferring input and output schema
    # Create sample input and predictions
    # sample_input = np.random.uniform(size=[1, 28, 28]).astype(np.float32)

    # # Get model output - convert tensor to numpy
    # with torch.no_grad():
    #     output = model(torch.tensor(sample_input))
    #     sample_output = output.numpy()

    # # Infer signature automatically
    # signature = infer_signature(sample_input, sample_output)
        
    # Save the trained model to MLflow.
    model_info = mlflow.pytorch.log_model(model, name="PyTorch_MLFlow_model", signature=signature)

Epoch 1-------------------------------
loss: 0.488221 accuracy: 0.796875 [0 / 938]
loss: 0.606702 accuracy: 0.734375 [100 / 938]
loss: 0.394706 accuracy: 0.843750 [200 / 938]
loss: 0.676098 accuracy: 0.750000 [300 / 938]
loss: 0.617312 accuracy: 0.671875 [400 / 938]
loss: 0.626373 accuracy: 0.796875 [500 / 938]
loss: 0.605164 accuracy: 0.734375 [600 / 938]
loss: 0.631529 accuracy: 0.781250 [700 / 938]
loss: 0.661463 accuracy: 0.765625 [800 / 938]
loss: 0.565829 accuracy: 0.750000 [900 / 938]
Eval metrics: Accuracy: 0.78, Avg loss: 0.591617 
Epoch 2-------------------------------
loss: 0.467491 accuracy: 0.812500 [0 / 938]
loss: 0.571770 accuracy: 0.734375 [100 / 938]
loss: 0.376837 accuracy: 0.875000 [200 / 938]
loss: 0.646474 accuracy: 0.750000 [300 / 938]
loss: 0.602310 accuracy: 0.718750 [400 / 938]
loss: 0.606144 accuracy: 0.781250 [500 / 938]
loss: 0.574763 accuracy: 0.765625 [600 / 938]
loss: 0.626924 accuracy: 0.812500 [700 / 938]
loss: 0.669579 accuracy: 0.765625 [800 / 938]
lo

In [18]:
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

In [19]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

# Add batch dimension to the input for prediction
input_data = np.expand_dims(training_data[0][0].numpy(), axis=0)
outputs = loaded_model.predict(input_data)
predicted, actual = classes[outputs[0].argmax()], classes[training_data[0][1]]
print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


# Complete PyTorch Logging Example

Earlier we saw a simple implementation of logging model training metrics on Databricks using CNNs implemented using PyTorch-MLFlow-Databricks. Below we will implement something similar, but using standard DNNs and we will log many more metrices and visualisations.  

In [7]:
#mlflow.create_experiment("/Users/debajyoti.das.bookworm@gmail.com/mlflow-pytorch-complete",artifact_location="dbfs:/Volumes/test/mlflow/mlflow-pytorch-complete")
mlflow.set_experiment("/Users/debajyoti.das.bookworm@gmail.com/mlflow-pytorch-complete-v2")

<Experiment: artifact_location='dbfs:/databricks/mlflow-tracking/1304458245524678', creation_time=1766882239452, experiment_id='1304458245524678', last_update_time=1766882394802, lifecycle_stage='active', name='/Users/debajyoti.das.bookworm@gmail.com/mlflow-pytorch-complete-v2', tags={'mlflow.experiment.sourceName': '/Users/debajyoti.das.bookworm@gmail.com/mlflow-pytorch-complete-v2',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'debajyoti.das.bookworm@gmail.com',
 'mlflow.ownerId': '8680600426295472'}>

In [8]:
# Define the model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(nn.Linear(28 * 28, 512),
                                               nn.ReLU(),
                                               nn.Linear(512, 512),
                                               nn.ReLU(),
                                               nn.Linear(512, 10))

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [9]:
class MLflowTracker:
    def __init__(self, model, classes):
        self.model = model
        self.classes = classes
        self.train_losses = []
        self.val_losses = []
        self.train_accs = []
        self.val_accs = []

    def log_epoch(self, epoch, train_loss, train_acc, val_loss, val_acc):
        """Log metrics for an epoch."""
        self.train_losses.append(train_loss)
        self.val_losses.append(val_loss)
        self.train_accs.append(train_acc)
        self.val_accs.append(val_acc)

        mlflow.log_metrics(
            {
                "train_loss": train_loss,
                "train_accuracy": train_acc,
                "val_loss": val_loss,
                "val_accuracy": val_acc,
            },
            step=epoch,
        )

    def log_confusion_matrix(self, val_loader, device):
        """Generate and log confusion matrix."""
        self.model.eval()
        all_preds = []
        all_targets = []

        with torch.no_grad():
            for inputs, targets in val_loader:
                inputs = inputs.to(device)
                targets = targets.to(device)
                outputs = self.model(inputs)
                _, preds = torch.max(outputs, 1)

                all_preds.extend(preds.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())

        # Create confusion matrix
        cm = confusion_matrix(all_targets, all_preds)

        # Plot
        plt.figure(figsize=(10, 8))
        sns.heatmap(
            cm,
            annot=True,
            fmt="d",
            cmap="Blues",
            xticklabels=self.classes,
            yticklabels=self.classes,
        )
        plt.title("Confusion Matrix")
        plt.ylabel("True Label")
        plt.xlabel("Predicted Label")
        plt.tight_layout()

        # Save and log
        plt.savefig("confusion_matrix.png")
        mlflow.log_artifact("confusion_matrix.png")
        plt.close()

    def log_training_curves(self):
        """Generate and log training curves."""
        plt.figure(figsize=(12, 5))

        # Loss subplot
        plt.subplot(1, 2, 1)
        plt.plot(self.train_losses, label="Train Loss")
        plt.plot(self.val_losses, label="Validation Loss")
        plt.title("Loss Curves")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.legend()

        # Accuracy subplot
        plt.subplot(1, 2, 2)
        plt.plot(self.train_accs, label="Train Accuracy")
        plt.plot(self.val_accs, label="Validation Accuracy")
        plt.title("Accuracy Curves")
        plt.xlabel("Epoch")
        plt.ylabel("Accuracy (%)")
        plt.legend()

        plt.tight_layout()
        plt.savefig("training_curves.png")
        mlflow.log_artifact("training_curves.png")
        plt.close()

In [11]:
# Training parameters
params = {"epochs": 3,
          "learning_rate": 1e-3,
          "batch_size": batch_size,
          "optimizer": "SGD",
          "model_type": "MLP",
          "hidden_units": [512, 512]}

# Create and prepare model
model = NeuralNetwork().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=params["learning_rate"])  

In [12]:
# Initialize tracker
tracker = MLflowTracker(
    model,
    classes=[
        "T-shirt",
        "Trouser",
        "Pullover",
        "Dress",
        "Coat",
        "Sandal",
        "Shirt",
        "Sneaker",
        "Bag",
        "Ankle boot",
    ],
)

In [17]:
# Training and logging
with mlflow.start_run():
    # 1. Log parameters
    mlflow.log_params(params)

    # 2. Log model architecture
    with open("model_fmnist_pt_mlf_cmplt_summary.txt", "w") as f:
        f.write(str(summary(model, input_size=(1, 1, 28, 28), device=device)))
    mlflow.log_artifact("model_fmnist_pt_mlf_cmplt_summary.txt")

    # 3. Training loop with metric logging
    for epoch in range(params["epochs"]):
        model.train()
        train_loss = 0
        correct = 0
        total = 0

        for batch_idx, (data, target) in enumerate(train_dataloader):
            data = data.to(device)
            target = target.to(device)

            # Forward pass
            optimizer.zero_grad()
            output = model(data)
            loss = loss_fn(output, target)

            # Backward pass
            loss.backward()
            optimizer.step()

            # Calculate metrics
            train_loss += loss.item()
            _, predicted = output.max(1)
            total += target.size(0)
            correct += predicted.eq(target).sum().item()

            # Log batch metrics (every 100 batches)
            if batch_idx % 100 == 0:
                batch_loss = train_loss / (batch_idx + 1)
                batch_acc = 100.0 * correct / total
                mlflow.log_metrics({"batch_loss": batch_loss, "batch_accuracy": batch_acc},
                                   step=epoch * len(train_dataloader) + batch_idx)
                
        # Calculate epoch metrics
        epoch_loss = train_loss / len(train_dataloader)
        epoch_acc = 100.0 * correct / total

        # Validation
        model.eval()
        val_loss = 0
        val_correct = 0
        val_total = 0

        with torch.no_grad():
            for data, target in test_dataloader:
                data = data.to(device)
                target = target.to(device)
                output = model(data)
                loss = loss_fn(output, target)

                val_loss += loss.item()
                _, predicted = output.max(1)
                val_total += target.size(0)
                val_correct += predicted.eq(target).sum().item()

        # Calculate and log epoch validation metrics
        val_loss = val_loss / len(test_dataloader)
        val_acc = 100.0 * val_correct / val_total

        # Log epoch metrics
        mlflow.log_metrics({"train_loss": epoch_loss,
                            "train_accuracy": epoch_acc,
                            "val_loss": val_loss,
                            "val_accuracy": val_acc}, step=epoch)
        # Log epoch metrics
        tracker.log_epoch(epoch, epoch_loss, epoch_acc, val_loss, val_acc)

        # Log final visualizations
        tracker.log_confusion_matrix(test_dataloader, device)
        tracker.log_training_curves()

        print(f"Epoch {epoch+1}/{params['epochs']}, "
              f"Train Loss: {epoch_loss:.4f}, Train Acc: {epoch_acc:.2f}%, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        
    # Create sample input and predictions
    sample_input = np.random.uniform(size=[1, 28, 28]).astype(np.float32)

    # Get model output - convert tensor to numpy
    with torch.no_grad():
        output = model(torch.tensor(sample_input).to(device))
        sample_output = output.cpu().numpy()

    # Infer signature automatically
    signature = infer_signature(sample_input, sample_output)
        
    # 5. Log the trained model
    model_info = mlflow.pytorch.log_model(model, name="model", signature=signature)

Epoch 1/3, Train Loss: 0.9403, Train Acc: 67.44%, Val Loss: 0.9160, Val Acc: 67.14%
Epoch 2/3, Train Loss: 0.8760, Train Acc: 68.72%, Val Loss: 0.8625, Val Acc: 68.00%
Epoch 3/3, Train Loss: 0.8283, Train Acc: 70.06%, Val Loss: 0.8217, Val Acc: 69.39%
🏃 View run bright-elk-717 at: https://dbc-e4fb7400-b637.cloud.databricks.com/ml/experiments/1304458245524678/runs/2b957c18247f44ebbf863aa673b546bb
🧪 View experiment at: https://dbc-e4fb7400-b637.cloud.databricks.com/ml/experiments/1304458245524678


In [18]:
# 6. Final evaluation
model.eval()
test_loss = 0
test_correct = 0
test_total = 0

with torch.no_grad():
    for data, target in test_dataloader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        loss = loss_fn(output, target)

        test_loss += loss.item()
        _, predicted = output.max(1)
        test_total += target.size(0)
        test_correct += predicted.eq(target).sum().item()

# Calculate and log final test metrics
test_loss = test_loss / len(test_dataloader)
test_acc = 100.0 * test_correct / test_total

mlflow.log_metrics({"test_loss": test_loss, "test_accuracy": test_acc})

print(f"Final Test Accuracy: {test_acc:.2f}%")

Final Test Accuracy: 69.39%


In [19]:
# Load and use the model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

# Make predictions
sample_input = np.random.uniform(size=[1, 28, 28]).astype(np.float32)
predictions = loaded_model.predict(sample_input)
print("Predictions:", predictions)

Predictions: [[ 0.7429988  -3.2293518   2.366237   -0.9774927   1.4614708  -0.17526606
   2.129318   -3.2427442   2.9796162  -1.5677054 ]]


In [25]:
mlflow.end_run()

🏃 View run stylish-goose-867 at: https://dbc-e4fb7400-b637.cloud.databricks.com/ml/experiments/1304458245524678/runs/5a3b626637b34854b29c19fa24510558
🧪 View experiment at: https://dbc-e4fb7400-b637.cloud.databricks.com/ml/experiments/1304458245524678


# Hyperparameter Optimization
Combine PyTorch with hyperparameter optimization tools while tracking everything in MLflow:

In [34]:
# Define the model
class NeuralNetworkOptim(nn.Module):
    def __init__(self, hidden_size):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(nn.Linear(28 * 28, out_features=hidden_size),
                                               nn.ReLU(),
                                               nn.Linear(hidden_size, hidden_size),
                                               nn.ReLU(),
                                               nn.Linear(hidden_size, 10))
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [35]:
def train_epoch(model, dataloader, loss_fn, optimizer, epoch, device):
    model.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (data, target) in enumerate(dataloader):
        data = data.to(device)
        target = target.to(device)

        # Forward pass
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)

        # Backward pass
        loss.backward()
        optimizer.step()

        # Calculate metrics
        train_loss += loss.item()
        _, predicted = output.max(1)
        total += target.size(0)
        correct += predicted.eq(target).sum().item()

        # Log batch metrics (every 100 batches)
        if batch_idx % 100 == 0:
            batch_loss = train_loss / (batch_idx + 1)
            batch_acc = 100.0 * correct / total
            mlflow.log_metrics({"batch_loss": batch_loss, "batch_accuracy": batch_acc},
                                step=epoch * len(dataloader) + batch_idx)
            
    # Calculate epoch metrics
    epoch_loss = train_loss / len(dataloader)
    epoch_acc = 100.0 * correct / total

    return epoch_loss, epoch_acc

In [36]:
def evaluate(model, dataloader, loss_fn, device):
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for data, target in dataloader:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            loss = loss_fn(output, target)

            val_loss += loss.item()
            _, predicted = output.max(1)
            val_total += target.size(0)
            val_correct += predicted.eq(target).sum().item()

    # Calculate and log epoch validation metrics
    val_loss = val_loss / len(dataloader)
    val_acc = 100.0 * val_correct / val_total

    return val_loss, val_acc

In [ ]:
def objective(trial, train_loader, val_loader, device):
    # Suggest hyperparameters
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD"])
    hidden_size = trial.suggest_categorical("hidden_size", [128, 256, 512])

    with mlflow.start_run(nested=True):
        # Log hyperparameters
        params = {"lr": lr,
                  "optimizer": optimizer_name,
                  "hidden_size": hidden_size,
                  "batch_size": 64,
                  "epochs": 3}
        mlflow.log_params(params)

        # Create model
        model = NeuralNetworkOptim(hidden_size=hidden_size).to(device)
        loss_fn = nn.CrossEntropyLoss()

        # Configure optimizer
        if optimizer_name == "Adam":
            optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        else:
            optimizer = torch.optim.SGD(model.parameters(), lr=lr)

        # Train for a few epochs
        best_val_acc = 0
        for epoch in range(params["epochs"]):
            # Training code (abbreviated)...
            train_loss, train_acc = train_epoch(model, train_loader, loss_fn, optimizer, epoch, device)
            val_loss, val_acc = evaluate(model, val_loader, loss_fn, device)

            mlflow.log_metrics({"train_loss": train_loss,
                                "train_acc": train_acc,
                                "val_loss": val_loss,
                                "val_acc": val_acc},
                                step=epoch)

            best_val_acc = max(best_val_acc, val_acc)

        # Final logging
        mlflow.log_metric("best_val_acc", best_val_acc)

        #Getting model signature and logging the model
        sample_input = np.random.uniform(size=[1, 28, 28]).astype(np.float32)
        # Get model output - convert tensor to numpy
        with torch.no_grad():
            output = model(torch.tensor(sample_input).to(device))
            sample_output = output.cpu().numpy()
        signature = infer_signature(sample_input, sample_output)
        
        mlflow.pytorch.log_model(model, name="model", signature=signature)

        return best_val_acc

In [40]:
# Execute hyperparameter search
with mlflow.start_run(run_name="hyperparam_optimization"):
    study = optuna.create_study(direction="maximize")
    objective_func = partial(objective, train_loader=train_dataloader, val_loader=test_dataloader, device=device)
    study.optimize(objective_func, n_trials=20)

    # Log best parameters and score
    mlflow.log_params(study.best_params)
    mlflow.log_metric("best_val_accuracy", study.best_value)

[I 2025-12-28 11:53:38,819] A new study created in memory with name: no-name-8fb035da-127f-44cf-95d1-82573f293234
[I 2025-12-28 11:54:09,725] Trial 0 finished with value: 23.19 and parameters: {'lr': 0.0002580915655688361, 'optimizer': 'SGD', 'hidden_size': 128}. Best is trial 0 with value: 23.19.


🏃 View run dazzling-snake-633 at: https://dbc-e4fb7400-b637.cloud.databricks.com/ml/experiments/1304458245524678/runs/d95508865f844a6187a3cede1471df91
🧪 View experiment at: https://dbc-e4fb7400-b637.cloud.databricks.com/ml/experiments/1304458245524678


[I 2025-12-28 11:54:39,776] Trial 1 finished with value: 79.91 and parameters: {'lr': 0.012758063061313727, 'optimizer': 'SGD', 'hidden_size': 512}. Best is trial 1 with value: 79.91.


🏃 View run abrasive-snake-997 at: https://dbc-e4fb7400-b637.cloud.databricks.com/ml/experiments/1304458245524678/runs/cf98c3ebfc8b44ee85fb3e7ef1d4e689
🧪 View experiment at: https://dbc-e4fb7400-b637.cloud.databricks.com/ml/experiments/1304458245524678


[I 2025-12-28 11:55:10,187] Trial 2 finished with value: 80.87 and parameters: {'lr': 0.03384407825078113, 'optimizer': 'SGD', 'hidden_size': 512}. Best is trial 2 with value: 80.87.


...

🏃 View run vaunted-conch-859 at: https://dbc-e4fb7400-b637.cloud.databricks.com/ml/experiments/1304458245524678/runs/f0ac3769f056439ea31689f2f1bac6b2
🧪 View experiment at: https://dbc-e4fb7400-b637.cloud.databricks.com/ml/experiments/1304458245524678


[I 2025-12-28 12:04:00,693] Trial 18 finished with value: 86.15 and parameters: {'lr': 0.0005398302726771358, 'optimizer': 'Adam', 'hidden_size': 512}. Best is trial 11 with value: 86.47.


🏃 View run receptive-skunk-329 at: https://dbc-e4fb7400-b637.cloud.databricks.com/ml/experiments/1304458245524678/runs/043da3f2232f4443838b844e235fe2f6
🧪 View experiment at: https://dbc-e4fb7400-b637.cloud.databricks.com/ml/experiments/1304458245524678


[I 2025-12-28 12:04:33,716] Trial 19 finished with value: 85.48 and parameters: {'lr': 0.0016393332505129647, 'optimizer': 'Adam', 'hidden_size': 256}. Best is trial 11 with value: 86.47.


🏃 View run secretive-trout-503 at: https://dbc-e4fb7400-b637.cloud.databricks.com/ml/experiments/1304458245524678/runs/2709e0b01bd14020a1e6a5a48462e516
🧪 View experiment at: https://dbc-e4fb7400-b637.cloud.databricks.com/ml/experiments/1304458245524678
🏃 View run hyperparam_optimization at: https://dbc-e4fb7400-b637.cloud.databricks.com/ml/experiments/1304458245524678/runs/acb255ca7ec1487abe1162e2393238b3
🧪 View experiment at: https://dbc-e4fb7400-b637.cloud.databricks.com/ml/experiments/1304458245524678
